파이토치의 nn.RNN으로 RNN 간단하게 구현해보기

RNN(순환 신경망)은 시퀀스 모델이다. 데이터가 한번에 다 들어가는 것이 아닌 시퀀스 단위로 처리하는 모델이다. 
시퀀스로 처리하게 되면 예를 들어서 "나는 정말 피곤하다." 라는 문장이 있을 때
1. 나는
2. 정말
3. 피곤하다
순서로 처리하게 되는데 이때 나는에 사용되는 가중치 W와 정말에 사용되는 W, 피곤하다에 사용되는 W가 다르게 된다. 

대략적으로 순서는 나는 정말 피곤하다를 각각 embedding해서 벡터로 표현하고, 이를 hidden layer에서 순차적으로 처리한다.
embedding(나는) * W ->  embedding(정말) * W + embedding(나는) * W --- 
이런식으로 진행된다.


In [1]:
import torch
import torch.nn as nn

input_size = 5
hidden_size = 8

#입력 텐서 정의 : (배치크기 * 시점의 수 * 매 시점마다 들어가는 입력)
inputs = torch.Tensor(1, 10 , 5)


In [4]:
cell = nn.RNN(input_size,hidden_size,batch_first = True)

outputs, _status = cell(inputs)
print(inputs)
print(outputs)


tensor([[[2.1324e-14, 2.0893e-42, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]]])
tensor([[[-0.4847, -0.1222,  0.5851,  0.0393,  0.2962, -0.3598, -0.0980,
           0.1506],
         [-0.6468, -0.1595,  0.3162, -0.1091,  0.4427, -0.3270, -0.1337,
          -0.1943],
         [-0.6665, -0.1499,  0.2470, -0.2614,  0.4655, -0.4559, -0.3292,
          -0.1864],
         

In [ ]:
#cell은 2개의 값을 return하게 되는데 첫번째는 모든 시점의 은닉상태들이고, 두번째는 마지막 시점의 은닉상태이다. 

RNN 모델은 깊게 쌓을 수 있다. 다음 time 뿐만 아니라 다음 layer에도 전달하게 된다.

In [5]:
inputs = torch.Tensor(1,10,5) # batch time inputsize
cell = nn.RNN(input_size=5, hidden_size=8,num_layers=2, batch_first=True)

output,_status = cell(inputs)
print(output)


tensor([[[-0.0065, -0.1013, -0.4842, -0.1214,  0.2630, -0.1605,  0.3212,
           0.1009],
         [-0.0424,  0.0148, -0.6516, -0.0543,  0.0528,  0.0638,  0.4792,
           0.2978],
         [ 0.1468, -0.0850, -0.7082,  0.0095, -0.0115,  0.1223,  0.4859,
           0.2777],
         [ 0.1072, -0.0542, -0.6750,  0.0455, -0.0238,  0.2401,  0.4979,
           0.2571],
         [ 0.1151, -0.0680, -0.6744,  0.0115, -0.0368,  0.2548,  0.5083,
           0.2826],
         [ 0.1227, -0.0807, -0.6739,  0.0095, -0.0303,  0.2497,  0.5152,
           0.2898],
         [ 0.1172, -0.0745, -0.6729,  0.0108, -0.0336,  0.2551,  0.5172,
           0.2905],
         [ 0.1210, -0.0765, -0.6736,  0.0105, -0.0331,  0.2537,  0.5170,
           0.2914],
         [ 0.1198, -0.0756, -0.6733,  0.0115, -0.0334,  0.2549,  0.5170,
           0.2908],
         [ 0.1203, -0.0757, -0.6733,  0.0112, -0.0334,  0.2549,  0.5171,
           0.2910]]], grad_fn=<TransposeBackward1>)


양방향 순환 신경망은 언어라는 것이 이전 시점 데이터 뿐만 아니라 미래에 있는 데이터 또한 불러와야 되기 때문에 필요하다.
RNN이 과거 시점의 데이터들을 참고해서 정답을 예측한다. 또한 향후에 시점의 데이터들도 참고한다.
두개의 메모리 셀을 두고 하나는 기본 RNN과 같이 다음 time step으로 보내고, 하나는 다음 time step에서 가져온다.


In [6]:
inputs = torch.Tensor(1,10,5)
cell = nn.RNN(5,8,2,batch_first=True, bidirectional=True) #bidirectional 을 True로 해주면 양방향이 적용된다.
output,_status = cell(inputs)
print(output)
#이때 output의 size는 2배가 된다 (미래로 보내는것과 미래에서 가져오는애들이 concat되어있음)

tensor([[[-0.0774, -0.1980,  0.3144,  0.2494,  0.3557, -0.1459,  0.4364,
          -0.5083,  0.0631, -0.4325, -0.4235,  0.4349,  0.2658, -0.0540,
           0.1977,  0.1565],
         [ 0.3184, -0.2619,  0.4942,  0.6167,  0.2611, -0.3365,  0.3204,
          -0.5484,  0.1817, -0.3334, -0.3389,  0.4242,  0.2895, -0.1975,
           0.1963,  0.1367],
         [ 0.3915, -0.4621,  0.4402,  0.5916,  0.2099, -0.3588,  0.1682,
          -0.6453,  0.1990, -0.3000, -0.3420,  0.4382,  0.2982, -0.1531,
           0.1744,  0.1401],
         [ 0.3322, -0.5013,  0.4158,  0.5072,  0.2760, -0.3235,  0.1136,
          -0.6745,  0.1874, -0.3216, -0.3487,  0.4357,  0.2803, -0.1541,
           0.1612,  0.1480],
         [ 0.3164, -0.4890,  0.4210,  0.4936,  0.2956, -0.3179,  0.1505,
          -0.6673,  0.1965, -0.3081, -0.3534,  0.4250,  0.2859, -0.1619,
           0.1661,  0.1486],
         [ 0.3296, -0.4743,  0.4304,  0.5139,  0.2858, -0.3165,  0.1575,
          -0.6633,  0.1879, -0.3133, -0.3414,  0.427